In [1]:
from web3 import Web3
import pandas as pd
import datetime
from concurrent.futures import ProcessPoolExecutor
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
#key_1 = "4d08ab7e29294be397e15ec5503401ca"
key_2 = "d4d8ddd18ef6438ea49fd3d5006b7eba"
key_3 = "b7b06ad6a7304e2197efa10b79e1c867"
key_4 = "58bce48fe80b451c96fee8ef35bbcce8"

provider = "https://mainnet.infura.io/v3/"

In [8]:
from web3 import Web3
import pandas as pd
from datetime import datetime
import numpy as np
from joblib import Parallel, delayed
import logging, os, json


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

data_dir = 'block_data'
pkl_file_path = os.path.join(data_dir, 'ethereum_transactions.pkl')

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

        
def save_transactions_to_pkl(block_number, transactions_data):
    """Process and save block transaction data directly to a .pkl file."""
    try:
        if os.path.exists(pkl_file_path):
            df_existing = pd.read_pickle(pkl_file_path)
        else:
            df_existing = pd.DataFrame()

        for transaction in transactions_data:
            if 'timestamp' in transaction and isinstance(transaction['timestamp'], datetime):
                transaction['timestamp'] = transaction['timestamp'].isoformat()
            
            for key, value in transaction.items():
                if isinstance(value, np.int64):
                    transaction[key] = int(value)
                    
        df_new_transactions = pd.DataFrame(transaction)

        df_combined = pd.concat([df_existing, df_new_transactions], ignore_index=True)

        df_combined.to_pickle(pkl_file_path)
        logging.info(f"Block {block_number} transactions added to the .pkl file.")
    except Exception as e:
        logging.error(f"Error processing transactions for block {block_number}: {e}")


def load_block_data(block_number):
    """Check if block transaction data exists as a .pkl file."""
    file_path = os.path.join(data_dir, f'{block_number}.json')
    if os.path.exists(file_path):
        print(f'Block {block_number} data already exists')
        return True
    return False


provider_alchemy = "https://eth-mainnet.g.alchemy.com/v2/lv9CnJ8RqVk9hp-KRG2X_RS8wx3bNJbY"

w3 = Web3(Web3.HTTPProvider(provider_alchemy)) # provider_infura + key_4))
start_block = 18938321
end_block   = 18993488


transactions_data = []
block_range = np.arange(start_block, end_block + 1)
filtered_block_range = [block for block in block_range if not load_block_data(block)]

print(len(filtered_block_range)/len(block_range)*100,"% of blocks are new")

for block_number in filtered_block_range:
    block_number_int = int(block_number)
    block = w3.eth.get_block(block_number_int, full_transactions=True)
    block_timestamp = datetime.utcfromtimestamp(block.timestamp)
    
    for tx in block.transactions:
        tx_receipt = w3.eth.get_transaction_receipt(tx.hash)
        
        transactions_data.append({
            'block_number': block_number,
            'tx_hash': tx.hash.hex(),
            'gas_price': tx.gasPrice,
            'gas_used': tx_receipt.gasUsed,
            'timestamp': block_timestamp
        })
    
    
    
    save_block_data(block_number, transactions_data)

print("Dataframe saved. Number of transactions:", len(transactions_data))

df_transactions = pd.DataFrame(transactions_data)

df_transactions['gas_price_gwei'] = df_transactions['gas_price'].apply(lambda x: w3.from_wei(x, 'gwei')).astype(int)
df_transactions['tx_hash']        = df_transactions['tx_hash'].astype(str)
df_transactions['timestamp']      = pd.to_datetime(df_transactions['timestamp'])
df_transactions = df_transactions.set_index('timestamp').sort_index()

df_transactions.to_pickle('ethereum_transactions.pkl')

Block 18935888 data already exists
Block 18935889 data already exists
Block 18935890 data already exists
Block 18935891 data already exists
Block 18935892 data already exists
Block 18935893 data already exists
Block 18935894 data already exists
Block 18935895 data already exists
Block 18935896 data already exists
Block 18935897 data already exists
Block 18935898 data already exists
Block 18935899 data already exists
Block 18935900 data already exists
Block 18935901 data already exists
Block 18935902 data already exists
Block 18935903 data already exists
Block 18935904 data already exists
Block 18935905 data already exists
Block 18935906 data already exists
Block 18935907 data already exists
Block 18935908 data already exists
Block 18935909 data already exists
Block 18935910 data already exists
Block 18935911 data already exists
Block 18935912 data already exists
Block 18935913 data already exists
Block 18935914 data already exists
Block 18935915 data already exists
Block 18935916 data 

2024-02-10 13:10:10,886 - INFO - Block 18937472 data saved.
2024-02-10 13:10:28,371 - INFO - Block 18937473 data saved.
2024-02-10 13:10:43,145 - INFO - Block 18937474 data saved.
2024-02-10 13:10:58,001 - INFO - Block 18937475 data saved.
2024-02-10 13:11:27,127 - INFO - Block 18937476 data saved.
2024-02-10 13:11:41,712 - INFO - Block 18937477 data saved.
2024-02-10 13:11:58,615 - INFO - Block 18937478 data saved.
2024-02-10 13:12:13,460 - INFO - Block 18937479 data saved.
2024-02-10 13:12:27,798 - INFO - Block 18937480 data saved.
2024-02-10 13:12:52,426 - INFO - Block 18937481 data saved.
2024-02-10 13:13:09,991 - INFO - Block 18937482 data saved.
2024-02-10 13:13:26,234 - INFO - Block 18937483 data saved.
2024-02-10 13:13:45,709 - INFO - Block 18937484 data saved.
2024-02-10 13:13:56,692 - INFO - Block 18937485 data saved.
2024-02-10 13:14:20,099 - INFO - Block 18937486 data saved.
2024-02-10 13:14:39,610 - INFO - Block 18937487 data saved.
2024-02-10 13:14:53,574 - INFO - Block 1

KeyboardInterrupt: 

In [1]:
import pandas as pd
import os
import json
from web3 import Web3

def process_json_file(file_path):
    """Read a JSON file and convert it to a DataFrame."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    df['gas_price_gwei'] = df['gas_price'].apply(lambda x: Web3.from_wei(x, 'gwei'))
    return df

def save_df_to_pkl(df, pkl_file_path):
    """Append a DataFrame to a .pkl file, creating the file if it doesn't exist."""
    if os.path.exists(pkl_file_path):
        df_existing = pd.read_pickle(pkl_file_path)
        df_combined = pd.concat([df_existing, df], ignore_index=True)
    else:
        df_combined = df
    df_combined.to_pickle(pkl_file_path)

directory = 'block_data'
pkl_file_path = os.path.join(directory, 'compiled_eth_tx.pkl')
batch_size = 5

all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
all_files.sort()  # Optional: sort files if you need to process them in a specific order

for i in range(0, batch_size): # range(0, len(all_files), batch_size)
    batch_files = all_files[i:i+batch_size]
    frames = [process_json_file(file_path) for file_path in batch_files]
    df_batch = pd.concat(frames, ignore_index=True)
    df_batch['timestamp'] = pd.to_datetime(df_batch['timestamp'])
    save_df_to_pkl(df_batch, pkl_file_path)
    print(f"Processed and saved batch {i//batch_size + 1}/{(len(all_files) - 1)//batch_size + 1}")

print("All data processed and saved.")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


EOFError: Ran out of input